In [4]:
from pandas import read_csv, DataFrame
from os import getcwd

In [2]:
##########################################################################################
def get_demo_data():
    my_file_dir = getcwd() + "\\demographics\\demo_cleaned.csv"
    df = read_csv(my_file_dir)
    return df

##########################################################################################
def get_form_data():
    my_file_dir = getcwd() + "\\questionnaire\\main_form_cleaned.csv"
    df = read_csv(my_file_dir)
    return df

##########################################################################################
def get_task_data():
    my_file_dir = getcwd() + "\\task_performance\\movement_time.csv"
    df = read_csv(my_file_dir)
    return df

In [5]:
##########################################################################################
def replace_outliers_within_group(df:DataFrame, group_columns:list, target_column:str):
    # Reset index to ensure group_columns are not in the index
    df = df.reset_index(drop=True)

    # Function to calculate and replace outliers within each group
    def replace(group):
        Q1 = group[target_column].quantile(0.25)
        Q3 = group[target_column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        mean_value = group[(group[target_column] >= lower_bound) & (group[target_column] <= upper_bound)][target_column].mean()
        group[target_column] = group[target_column].apply(lambda x: mean_value if x < lower_bound or x > upper_bound else x)
        return group

    # Apply the function to each subgroup
    return df.groupby(group_columns).apply(replace).reset_index(drop=True)

In [12]:
demo_df = get_demo_data()

form_df = get_form_data()

task_df = get_task_data()


# merge demographics with questionnaire dataframe
merged_form_df = form_df.merge(demo_df)
# cols_not_to_check = ['pid','condition']
# print(merged_form_df)
dest_path1 = getcwd() + "\\merged_form_data.csv"
merged_form_df.to_csv(dest_path1, index=False)
print(" Successfully written merged questionnaire data to csv file! \n")


# merge demographics with task performance dataframe
merged_task_df = task_df.merge(demo_df)
cols_to_check = ['average_mt', 'average_skip_first_mt']
############ loop through all columns of dataframe and remove outliers ############
for col in list(merged_task_df.columns):
    if col in cols_to_check:
        print("Removing outliers for column = %s" % col)
        merged_task_df = replace_outliers_within_group(merged_task_df, ['condition', 'trial'], col)

dest_path2 = getcwd() + "\\merged_performance_data.csv"
merged_task_df.to_csv(dest_path2, index=False)
print(" Successfully written merged task performance data to csv file! \n")

 Successfully written merged questionnaire data to csv file! 

Removing outliers for column = average_mt
Removing outliers for column = average_skip_first_mt
 Successfully written merged task performance data to csv file! 

